*** Contexte : ***
Vous travaillez pour une startup en e-santé qui doit analyser des milliers de comptes-rendus
hospitaliers.
Ces documents contiennent un mélange de:
  1. Terminologie médicale spécialisée
  2. Abréviations courantes (ex: "BID", "TDM")
  3. Erreurs de transcription (ex: "hyp0glycémie")
  4. Notes manuscrites numérisées (ex: "pat. ++fébrile")

In [41]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string

# Télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Exercice 1: Nettoyage

In [42]:
def nettoyage(texte):
  # Convertir en minuscules
  texte = texte.lower()

  # Supprimer les dates (formats comme 12/11/23 ou 01-01-2024)
  texte = re.sub(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', '', texte)

  # Supprimer les valeurs numériques avec unités (ex: 125mg/L, 37 °c)
  texte = re.sub(r'\b\d+(?:\.\d+)?\s?[a-zA-Z°/%]+(?:/[a-zA-Z°%]+)?\b', '', texte)

  # Supprimer les caractères spéciaux spécifiques
  texte = re.sub(r'[°#]', '', texte)

  # Supprimer la ponctuation standard
  texte = texte.translate(str.maketrans('', '', string.punctuation))

  # Normaliser les abréviations et fautes courantes
  remplacements = {
      r'\bttt\b': 'traitement',
      r'\bpat\b\.?': 'patient',
  }

  for pattern, remplacement in remplacements.items():
      texte = re.sub(pattern, remplacement, texte)

  # Supprimer les chiffres isolés restants
  texte = re.sub(r'\d+', '', texte)

  # Supprimer les espaces multiples et les sauts de ligne
  texte = re.sub(r'\s+', ' ', texte).strip()

  return texte

### Exercice 2: Tokenisation Médicale

In [43]:
# Dictionnaire des abréviations médicales
abreviations = {
    "bid": "bis in die",
    "iv": "intraveineuse",
    "ttt": "traitement",
    "crp": "protéine c réactive",
    "tdm": "tomodensitométrie"
}

def tokeniser(texte):
    # Tokenisation spéciale pour les termes médicaux composés et les doses
    tokens = []

    # D'abord, on sépare les tokens avec les règles spéciales
    for token in word_tokenize(texte.lower()):
        # 1. Traitement des abréviations
        if token in abreviations:
            tokens.append(abreviations[token])
            continue

        # Keep COVID-19 (with hyphen)
        elif '-' in token and not token.startswith(('++', '--')):
            tokens.append(token)

        # Split doses (500mg/J → 500, mg, /, J)
        elif re.match(r'\d+[a-z]+(/[a-z]+)?', token, re.IGNORECASE):
            num = re.match(r'\d+', token).group()
            unit = token[len(num):]
            tokens.extend([num] + re.split(r'([/])', unit))
        else:
            tokens.append(token)

    return tokens

### Exercice 3: Normalisation Expert

In [44]:
termes_medicaux = {
    "corticoides": "glucocorticoides",
    "fébrile": "hyperthermique",
    "++fébrile": "hyperthermique",
    "azithromycine": "antibiotique macrolide"
}

corrections_ortho = {
    "hyp0glycémie": "hypoglycémie",
    "hyponatrémie": "hyponatrémie",
    "hyp0natrémie": "hyponatrémie",
    "fèbrile": "fébrile",
    "corticoïdes": "glucocorticoides",
    "corticoides": "glucocorticoides"
}

def normaliser(tokens):
    tokens_normalises = []

    for token in tokens:
        # 1. Correction orthographique d'abord
        token_corrige = corrections_ortho.get(token.lower(), token)

        # Correction générique des chiffres dans les mots (0 → o)
        if '0' in token_corrige:
            token_corrige = token_corrige.replace('0', 'o')

        # 2. Normalisation des termes médicaux
        if token_corrige in termes_medicaux:
            token_corrige = termes_medicaux[token_corrige]

        # 3. Normalisation des expressions avec ++
        elif token_corrige.startswith('++'):
            base = token_corrige[2:]
            if base in termes_medicaux:
                token_corrige = termes_medicaux[base]
            else:
                token_corrige = "hyper" + base

        tokens_normalises.append(token_corrige)

    return tokens_normalises

### Fonction pour tester le pipeline complet

In [48]:
# Fonction pour tester le pipeline complet
def teste_function(texte):
    texte_nettoye = nettoyage(texte)
    tokens = tokeniser(texte_nettoye)
    tokens_normalises = normaliser(tokens)

    # Supprimer les stopwords
    stop_words = set(stopwords.words('french'))
    tokens_filtres = [token for token in tokens_normalises if token not in stop_words]

    return tokens_filtres

### Test

In [49]:
texte_medical = """
  Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
  CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
  Ttt par corticoïdes IV + azithromycine 500mg/J.
  #Attention: allergie à la pénicilline!
  """

print(texte_medical)


  Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
  CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
  Ttt par corticoïdes IV + azithromycine 500mg/J.
  #Attention: allergie à la pénicilline!
  


In [50]:
# Nettoyer le texte
texte_nettoye = teste_function(texte_medical)
print("Texte nettoyé :", texte_nettoye)

Texte nettoyé : ['patient', 'admis', 'covid', 'sévère', 'protéine c réactive', 'fièvre', 'traitement', 'glucocorticoides', 'intraveineuse', 'antibiotique macrolide', 'attention', 'allergie', 'pénicilline']


### Test 2:

In [51]:
# Génération de 5 tests supplémentaires
test_cases = [
    """
    Patient 32 ans admis le 15-04-24 pour pneumonie bilatérale.
    Température: 38.5°C, SatO2: 92% en air ambiant.
    Prescrit: amoxicilline 1g BID + paracétamol 1g QID.
    """,
    """
    Femme 28 ans, grossesse 32SA, hospitalisée pour pyélonéphrite.
    Traitement: ceftriaxone IV 1g/J + gentamicine 240mg/J.
    Allergie: pénicilline (rash cutané).
    """,
    """
    Homme 60 ans, diabétique, ++polydipsie et polyurie.
    Glycémie capillaire: 3.2 mmol/L (hypoglicémie).
    Ttt: glucosé 30% 100ml IV.
    """,
    """
    Enfant 5 ans, fébrile à 40.1°C depuis 2 jours.
    Diagnostic: grippe A confirmée par TDM.
    Traitement: oseltamivir 45mg BID.
    """,
    """
    Pat. 70 ans, ATCD d'HTA, consulté pour ++dyspnée.
    ECG: FA rapide à 130/min. Traitement: digoxine 0.25mg/J.
    """
]

for i, test in enumerate(test_cases, 1):
    print(f"\nTest {i}:")
    print("Texte original:", test)
    result = teste_function(test)
    print("Résultat:", result)


Test 1:
Texte original: 
    Patient 32 ans admis le 15-04-24 pour pneumonie bilatérale.
    Température: 38.5°C, SatO2: 92% en air ambiant.
    Prescrit: amoxicilline 1g BID + paracétamol 1g QID.
    
Résultat: ['patient', 'admis', 'pneumonie', 'bilatérale', 'température', 'sato', 'air', 'ambiant', 'prescrit', 'amoxicilline', 'bis in die', 'paracétamol', 'qid']

Test 2:
Texte original: 
    Femme 28 ans, grossesse 32SA, hospitalisée pour pyélonéphrite.
    Traitement: ceftriaxone IV 1g/J + gentamicine 240mg/J.
    Allergie: pénicilline (rash cutané).
    
Résultat: ['femme', 'grossesse', 'hospitalisée', 'pyélonéphrite', 'traitement', 'ceftriaxone', 'intraveineuse', 'gentamicine', 'allergie', 'pénicilline', 'rash', 'cutané']

Test 3:
Texte original: 
    Homme 60 ans, diabétique, ++polydipsie et polyurie.
    Glycémie capillaire: 3.2 mmol/L (hypoglicémie).
    Ttt: glucosé 30% 100ml IV.
    
Résultat: ['homme', 'diabétique', 'polydipsie', 'polyurie', 'glycémie', 'capillaire', 'hypogli